Кабанова Е. R42951  
Лабораторная работа №2  
Вот что значит плохое среднее  

In [45]:
import numpy as np
import matplotlib.pyplot as plt

In [46]:
base = 10 # параметр, может принимать любые целые значения > 1

def exact_sum(K):
    """Точное значение суммы всех элементов."""
    return 1.

def samples(K):
    """"Элементы выборки"."""
    # создаем K частей из base^k одинаковых значений
    parts = [np.full((base**k,), float(base)**(-k)/K) for k in range(0, K)] 
    # создаем выборку объединяя части
    samples = np.concatenate(parts) 
    # перемешиваем элементы выборки и возвращаем
    return np.random.permutation(samples)

def direct_sum(x):
    """Последовательная сумма всех элементов вектора x"""
    s = 0.
    for e in x: 
        s += e
    return s

def number_of_samples(K):
    """Число элементов в выборке"""
    return np.sum([base**k for k in range(0, K)])

def exact_mean(K):
    """Значение среднего арифметического по выборке с близкой к машинной точностью."""
    return 1./number_of_samples(K)

def exact_variance(K):
    """Значение оценки дисперсии с близкой к машинной точностью."""
    # разные значения элементов выборки
    values = np.asarray([float(base)**(-k)/K for k in range(0, K)], dtype=np.double)
    # сколько раз значение встречается в выборке
    count = np.asarray([base**k for k in range(0, K)])
    return np.sum(count*(values-exact_mean(K))**2)/number_of_samples(K)

In [47]:
K = 7 # число слагаемых
x = samples(K) # сохраняем выборку в массив
print("Число элементов:", len(x))
print("Самое маленькое и большое значения:", np.min(x), np.max(x))

exact_sum_for_x = exact_sum(K) # значение суммы с близкой к машинной погрешностью
direct_sum_for_x = direct_sum(x) # сумма всех элементов по порядку

def relative_error(x0, x):
    """Погрешность x при точном значении x0"""
    return np.abs(x0-x)/np.abs(x)

print("Погрешность прямого суммирования:", relative_error(exact_sum_for_x, direct_sum_for_x))

Число элементов: 1111111
Самое маленькое и большое значения: 1.4285714285714285e-07 0.14285714285714285
Погрешность прямого суммирования: 1.827538120802071e-11


In [48]:
sorted_x=x[np.argsort(x)]
sorted_sum_for_x=direct_sum(sorted_x)
print("Погрешность суммирования по возрастанию:", relative_error(exact_sum_for_x, sorted_sum_for_x))

Погрешность суммирования по возрастанию: 1.0016432128178195e-12


In [49]:
sorted_x=x[np.argsort(x)[::-1]]
sorted_sum_for_x=direct_sum(sorted_x)
print("Погрешность суммирования по убыванию:", relative_error(exact_sum_for_x, sorted_sum_for_x))

Погрешность суммирования по убыванию: 3.864975006557192e-11


In [50]:
def Kahan_sum(x):
    s = 0.0 # частичная сумма
    c = 0.0 # сумма погрешностей
    for i in x:
        y = i - c      # первоначально y равно следующему элементу последовательности
        t = s + y      # сумма s может быть велика, поэтому младшие биты y будут потеряны
        c = (t - s) - y  # (t-s) отбрасывает старшие биты, вычитание y восстанавливает младшие биты
        s = t        # новое значение старших битов суммы
    return s

Kahan_sum_for_x = Kahan_sum(x) # сумма всех элементов по порядку
print("Погрешность суммирования по Кэхэну:", relative_error(exact_sum_for_x, Kahan_sum_for_x))

Погрешность суммирования по Кэхэну: 0.0


In [51]:
# параметры выборки
mean = 1e6 # среднее
delta = 1e-5 # величина отклонения от среднего

def samples(N_over_two):
    """Генерирует выборку из 2*N_over_two значений с данным средним и среднеквадратическим 
    отклонением."""
    x = np.full((2*N_over_two,), mean, dtype=np.double)
    x[:N_over_two] += delta
    x[N_over_two:] -= delta
    return np.random.permutation(x)

def exact_mean():
    """Значение среднего арифметического по выборке с близкой к машинной точностью."""
    return mean

def exact_variance():
    """Значение оценки дисперсии с близкой к машинной точностью."""
    return delta**2

x = samples(1000000)

In [35]:
print("Размер выборки:", len(x))
print("Среднее значение:", exact_mean())
print("Оценка дисперсии:", exact_variance())
print("Ошибка среднего для встроенной функции:",relative_error(exact_mean(),np.mean(x)))
print("Ошибка дисперсии для встроенной функции:",relative_error(exact_variance(),np.var(x)))

Размер выборки: 2000000
Среднее значение: 1000000.0
Оценка дисперсии: 1.0000000000000002e-10
Ошибка среднего для встроенной функции: 0.0
Ошибка дисперсии для встроенной функции: 8.054126276826362e-06


In [52]:
def direct_mean(x):
    """Среднее через последовательное суммирование."""
    return direct_sum(x)/len(x)

print("Ошибка среднего для последовательного суммирования:",relative_error(exact_mean(),direct_mean(x)))

Ошибка среднего для последовательного суммирования: 2.444721758365625e-15


In [53]:
def direct_second_var(x):
    """Вторая оценка дисперсии через последовательное суммирование."""
    return direct_mean(x**2) - direct_mean(x)**2

def online_second_var(x):
    """Вторая оценка дисперсии через один проход по выборке"""
    m = x[0] # накопленное среднее 
    m2 = x[0]**2 # накопленное среднее квадратов
    for n in range(1, len(x)):
        m = (m * (n - 1) + x[n])/n
        m2 = (m2 * (n - 1) + x[n]**2)/n
    return m2 - m**2

print("Ошибка второй оценки дисперсии для последовательного суммирования:",relative_error(exact_variance(),direct_second_var(x)))
print("Ошибка второй оценки дисперсии для однопроходного суммирования:",relative_error(exact_variance(),online_second_var(x)))

Ошибка второй оценки дисперсии для последовательного суммирования: 0.9999999887780823
Ошибка второй оценки дисперсии для однопроходного суммирования: 0.9999999917252526


In [54]:
import math
def exact_sum_sin(K):
    return .5 * (np.sin(K) - np.cos(.5) / np.sin(.5) * np.cos(K) + np.cos(.5) / np.sin(.5)) / K

def samples_sin(N):
    global x, y
    samples_sin=[np.sin(i)/N for i in range(1, N+1)]
    y = sorted(samples_sin, key=abs)  
    x = np.random.permutation(samples_sin)

In [55]:
K = 7

direct = []
Kahan = []
sorted_data = []

for i in range(10):
    x = samples(K)
    exact_sum_for_x = exact_sum(K) # значение суммы с близкой к машинной погрешностью
    direct_sum_for_x = direct_sum(x) # сумма всех элементов по порядку
    sorted_x = x[np.argsort(x)]
    sorted_sum_for_x = direct_sum(sorted_x)
    Kahan_sum_for_x = Kahan_sum(x)
    
    direct.append(relative_error(exact_sum_for_x, direct_sum_for_x))
    Kahan.append(relative_error(exact_sum_for_x, Kahan_sum_for_x))
    sorted_data.append(relative_error(exact_sum_for_x, sorted_sum_for_x))
    
forward = sum(direct)/len(direct)
sorted_alg = sum(sorted_data)/len(sorted_data)
Kahan_alg = sum(Kahan)/len(Kahan)
    
print("По порядку", forward)
print("Сортировка по возрастанию", sorted_alg)
print("Алгоритм Кэхэна", Kahan_alg)

По порядку 0.9999999285714287
Сортировка по возрастанию 0.9999999285714287
Алгоритм Кэхэна 0.9999999285714287


In [56]:
step = 20
N = 1000000

x=[]
y=[]

direct = []
Kahan = []
Kahan_sorted = []
Kahan_sorted_abs = []
sorted_data = []

for i in range(step):    
    samples_sin(N)
    
    exact_sum_for_x=exact_sum_sin(N) # значение суммы с близкой к машинной погрешностью    
    direct_sum_for_x=direct_sum(x) # сумма всех элементов по порядку
    Kahan_sum_for_x=Kahan_sum(x)
    Kahan_sorted_x = Kahan_sum(sorted_x)
    Kahan_sum_abs_sorted=Kahan_sum(y)
    sorted_x=x[np.argsort(x)]
    sorted_sum_for_x=direct_sum(sorted_x)

    direct.append(relative_error(exact_sum_for_x, direct_sum_for_x))
    sorted_data.append(relative_error(exact_sum_for_x, sorted_sum_for_x))
    Kahan.append(relative_error(exact_sum_for_x, Kahan_sum_for_x))    
    Kahan_sorted.append(relative_error(exact_sum_for_x, Kahan_sorted_x))
    Kahan_sorted_abs.append(relative_error(exact_sum_for_x, Kahan_sum_abs_sorted))
    
Kahan_alg = sum(Kahan)/len(Kahan)
Kahan_alg_sorted = sum(Kahan_sorted)/len(Kahan_sorted)
Kahan_alg_sorted_abs = sum(Kahan_sorted_abs)/len(Kahan_sorted_abs)
sorted_alg = sum(sorted_data)/len(sorted_data)
forward = sum(direct)/len(direct)

print("По порядку", forward)
print("Сортировка по возрастанию", sorted_alg)
print("Алгоритм Кэхэна", Kahan_alg)
print("Алгоритм Кэхэна с сортировкой по возрастанию", Kahan_alg_sorted)
print("Алгоритм Кэхэна с сортировкой по возрастанию абсолютных значений", Kahan_alg_sorted_abs)

По порядку 1.2123600479694572e-10
Сортировка по возрастанию 7.98980699508693e-09
Алгоритм Кэхэна 2.5180409699682606e-13
Алгоритм Кэхэна с сортировкой по возрастанию 0.05000000000017677
Алгоритм Кэхэна с сортировкой по возрастанию абсолютных значений 1.856802348933412e-13
